# [Sentence-BERT](https://arxiv.org/pdf/1908.10084.pdf)

[Reference Code](https://www.pinecone.io/learn/series/nlp/train-sentence-transformers-softmax/)

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00


In [17]:
import os
import math
import re
from   random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
device

'cpu'

## 1. Data

### Train, Test, Validation

In [2]:
import datasets

mnli = datasets.load_dataset('glue', 'mnli')
mnli['train'].features

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [3]:
# List of datasets to remove 'idx' column from
mnli.column_names.keys()

dict_keys(['train', 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched'])

In [4]:
# Remove 'idx' column from each dataset
for column_names in mnli.column_names.keys():
    mnli[column_names] = mnli[column_names].remove_columns('idx')

In [5]:
mnli.column_names.keys()

dict_keys(['train', 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched'])

In [6]:
import numpy as np

np.unique(mnli['train']['label'])

array([0, 1, 2])

In [7]:
# Check the number of rows in each split of MNLI
print(f"Train set size: {mnli['train'].num_rows}")
print(f"Matched validation set size: {mnli['validation_matched'].num_rows}")
print(f"Mismatched validation set size: {mnli['validation_mismatched'].num_rows}")
print(f"Matched test set size: {mnli['test_matched'].num_rows}")
print(f"Mismatched test set size: {mnli['test_mismatched'].num_rows}")

Train set size: 392702
Matched validation set size: 9815
Mismatched validation set size: 9832
Matched test set size: 9796
Mismatched test set size: 9847


In [8]:
from datasets import DatasetDict

# Use only MNLI dataset
raw_dataset = DatasetDict({
    'train': mnli['train'].shuffle(seed=55).select(list(range(1000))),
    'test': mnli['test_mismatched'].shuffle(seed=55).select(list(range(1000))),
    'validation': mnli['validation_mismatched'].shuffle(seed=55).select(list(range(1000)))
})
#remove .select(list(range(1000))) in order to use full dataset
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 1000
    })
})

## 2. Preprocessing

In [9]:
# from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
class CustomBERTTokenizer:
  def __init__(self, word2id):
      self.word2id = word2id
      self.id2word = {v: k for k, v in self.word2id.items()}
      self.vocab_size = len(self.word2id)
      self.max_len = max_len

  def encode(self, sentences):
      output = {}
      output['input_ids'] = []
      output['attention_mask'] = []
      for sentence in sentences:
          input_ids = [self.word2id.get(word, self.word2id['[UNK]']) for word in sentence.split()]
          n_pad = self.max_len - len(input_ids)
          input_ids.extend([0] * n_pad)
          att_mask = [1 if idx != 0 else 0 for idx in input_ids]
          output['input_ids'].append(torch.tensor(input_ids))
          output['attention_mask'].append(torch.tensor(att_mask))
      return output

  def decode(self, ids):
      return ' '.join([self.id2word.get(idx.item(), '[UNK]') for idx in ids])

In [13]:
import pickle

bert_args = pickle.load(open('models/bert_args.pkl', 'rb'))
vocab_size = bert_args['vocab_size']
word2id = bert_args['word2id']
batch_size = bert_args['batch_size']
max_mask = bert_args['max_mask']
max_len = bert_args['max_len']
n_layers = bert_args['n_layers']
n_heads = bert_args['n_heads']
d_model = bert_args['d_model']
d_ff = bert_args['d_ff']
d_k = bert_args['d_k']
d_v = bert_args['d_v']
n_segments = bert_args['n_segments']

In [14]:
tokenizer = CustomBERTTokenizer(word2id)

In [18]:
def preprocess_function(examples):
    # Tokenize the premise
    premise_result = tokenizer.encode(
        examples['premise'])
    #num_rows, max_seq_length
    # Tokenize the hypothesis
    hypothesis_result = tokenizer.encode(
        examples['hypothesis'])
    #num_rows, max_seq_length
    # Extract labels
    labels = examples["label"]
    #num_rows
    return {
        "premise_input_ids": premise_result["input_ids"],
        "premise_attention_mask": premise_result["attention_mask"],
        "hypothesis_input_ids": hypothesis_result["input_ids"],
        "hypothesis_attention_mask": hypothesis_result["attention_mask"],
        "labels" : labels
    }

tokenized_datasets = raw_dataset.map(
    preprocess_function,
    batched=True,
)

tokenized_datasets = tokenized_datasets.remove_columns(['premise','hypothesis','label'])
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['premise_input_ids', 'premise_attention_mask', 'hypothesis_input_ids', 'hypothesis_attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['premise_input_ids', 'premise_attention_mask', 'hypothesis_input_ids', 'hypothesis_attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['premise_input_ids', 'premise_attention_mask', 'hypothesis_input_ids', 'hypothesis_attention_mask', 'labels'],
        num_rows: 1000
    })
})

## 3. Data loader

In [19]:
# torch.cuda.empty_cache()  # Frees unused GPU memory
# torch.cuda.memory_summary(device=None, abbreviated=False)  # Prints memory usage

In [20]:
from torch.utils.data import DataLoader

# initialize the dataloader
batch_size = 16 # due to cuda out of memory issue
train_dataloader = DataLoader(
    tokenized_datasets['train'],
    batch_size=batch_size,
    shuffle=True
)
eval_dataloader = DataLoader(
    tokenized_datasets['validation'],
    batch_size=batch_size
)
test_dataloader = DataLoader(
    tokenized_datasets['test'],
    batch_size=batch_size
)

In [21]:
for batch in train_dataloader:
    print(batch['premise_input_ids'].shape)
    print(batch['premise_attention_mask'].shape)
    print(batch['hypothesis_input_ids'].shape)
    print(batch['hypothesis_attention_mask'].shape)
    print(batch['labels'].shape)
    break

torch.Size([16, 1000])
torch.Size([16, 1000])
torch.Size([16, 1000])
torch.Size([16, 1000])
torch.Size([16])


## 4. Model

In [22]:
# start from a pretrained bert-base-uncased model
# from transformers import BertTokenizer, BertModel
# model = BertModel.from_pretrained('bert-base-uncased')
# model.to(device)

In [22]:
# embedding
class Embedding(nn.Module):
    def __init__(self, vocab_size, max_len, n_segments, d_model, device):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_len, d_model)      # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)
        self.device = device

    def forward(self, x, seg):
        #x, seg: (bs, len)
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long).to(self.device)
        pos = pos.unsqueeze(0).expand_as(x)  # (len,) -> (bs, len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

In [23]:
# scaled dot attention
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k, device):
        super(ScaledDotProductAttention, self).__init__()
        self.scale = torch.sqrt(torch.FloatTensor([d_k])).to(device)

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / self.scale # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

In [24]:
# multihead attention
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_model, d_k, device):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_k
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, self.d_v * n_heads)
        self.device = device
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention(self.d_k, self.device)(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(self.n_heads * self.d_v, self.d_model, device=self.device)(context)
        return nn.LayerNorm(self.d_model, device=self.device)(output + residual), attn # output: [batch_size x len_q x d_model]

In [25]:
# poswisefeedforwardnet
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(F.gelu(self.fc1(x)))

In [26]:
# encoder
class EncoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, d_ff, d_k, device):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(n_heads, d_model, d_k, device)
        self.pos_ffn       = PoswiseFeedForwardNet(d_model, d_ff)

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

In [27]:
# attention mask
def get_attn_pad_mask(seq_q, seq_k, device):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1).to(device)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

In [28]:
class BERT(nn.Module):
    def __init__(self, n_layers, n_heads, d_model, d_ff, d_k, n_segments, vocab_size, max_len, device):
        super(BERT, self).__init__()
        self.params = {'n_layers': n_layers, 'n_heads': n_heads, 'd_model': d_model,
                       'd_ff': d_ff, 'd_k': d_k, 'n_segments': n_segments,
                       'vocab_size': vocab_size, 'max_len': max_len}
        self.embedding = Embedding(vocab_size, max_len, n_segments, d_model, device)
        self.layers = nn.ModuleList([EncoderLayer(n_heads, d_model, d_ff, d_k, device) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))
        self.device = device

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids, self.device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]

        # 1. predict next sentence
        # it will be decided by first token(CLS)
        h_pooled   = self.activ(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_nsp = self.classifier(h_pooled) # [batch_size, 2]

        # 2. predict the masked token
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked  = self.norm(F.gelu(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_nsp, output

    def get_last_hidden_state(self, input_ids, segment_ids):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids, self.device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)

        return output

In [29]:
model = BERT(
    n_layers,
    n_heads,
    d_model,
    d_ff,
    d_k,
    n_segments,
    vocab_size,
    max_len,
    device
)
model.load_state_dict(torch.load('models/bert_model.pt'))
model.to(device)

<ipython-input-29-93cff7838cbc>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/bert_model.pt'))


BERT(
  (embedding): Embedding(
    (tok_embed): Embedding(23069, 768)
    (pos_embed): Embedding(1000, 768)
    (seg_embed): Embedding(2, 768)
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (enc_self_attn): MultiHeadAttention(
        (W_Q): Linear(in_features=768, out_features=512, bias=True)
        (W_K): Linear(in_features=768, out_features=512, bias=True)
        (W_V): Linear(in_features=768, out_features=512, bias=True)
      )
      (pos_ffn): PoswiseFeedForwardNet(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
      )
    )
  )
  (fc): Linear(in_features=768, out_features=768, bias=True)
  (activ): Tanh()
  (linear): Linear(in_features=768, out_features=768, bias=True)
  (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (classifier): Linear(in_features=768, out_features=2, bias=True)
  (deco

### Pooling
SBERT adds a pooling operation to the output of BERT / RoBERTa to derive a fixed sized sentence embedding

In [30]:
# define mean pooling function
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(
        token_embeds.size()
    ).float()
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool

## 5. Loss Function

## Classification Objective Function
We concatenate the sentence embeddings $u$ and $v$ with the element-wise difference  $\lvert u - v \rvert $ and multiply the result with the trainable weight  $ W_t ∈  \mathbb{R}^{3n \times k}  $:

$ o = \text{softmax}\left(W^T \cdot \left(u, v, \lvert u - v \rvert\right)\right) $

where $n$ is the dimension of the sentence embeddings and k the number of labels. We optimize cross-entropy loss. This structure is depicted in Figure 1.

## Regression Objective Function.
The cosine similarity between the two sentence embeddings $u$ and $v$ is computed (Figure 2). We use means quared-error loss as the objective function.

(Manhatten / Euclidean distance, semantically  similar sentences can be found.)

<img src="https://github.com/MyaMjechal/nlp-a4-do-you-agree-web-with-bert/blob/main/figures/sbert-architecture.png?raw=1" >

In [31]:
def configurations(u,v):
    # build the |u-v| tensor
    uv = torch.sub(u, v)   # batch_size,hidden_dim
    uv_abs = torch.abs(uv) # batch_size,hidden_dim

    # concatenate u, v, |u-v|
    x = torch.cat([u, v, uv_abs], dim=-1) # batch_size, 3*hidden_dim
    return x

def cosine_similarity(u, v):
    dot_product = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    similarity = dot_product / (norm_u * norm_v)
    return similarity

<img src="https://github.com/MyaMjechal/nlp-a4-do-you-agree-web-with-bert/blob/main/figures/sbert-ablation.png?raw=1" width="350" height="300">

In [32]:
classifier_head = torch.nn.Linear(768*3, 3).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
optimizer_classifier = torch.optim.Adam(classifier_head.parameters(), lr=2e-5)

criterion = nn.CrossEntropyLoss()

In [33]:
from transformers import get_linear_schedule_with_warmup

# and setup a warmup for the first ~10% steps
total_steps = int(len(raw_dataset) / batch_size)
warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(
		optimizer, num_warmup_steps=warmup_steps,
  	num_training_steps=total_steps - warmup_steps
)

# then during the training loop we update the scheduler per step
scheduler.step()

scheduler_classifier = get_linear_schedule_with_warmup(
		optimizer_classifier, num_warmup_steps=warmup_steps,
  	num_training_steps=total_steps - warmup_steps
)

# then during the training loop we update the scheduler per step
scheduler_classifier.step()

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


## 6. Training

In [34]:
from tqdm.auto import tqdm

num_epoch = 5
# 1 epoch should be enough, increase if wanted
for epoch in range(num_epoch):
    model.train()
    classifier_head.train()
    # initialize the dataloader loop with tqdm (tqdm == progress bar)
    for step, batch in enumerate(tqdm(train_dataloader, leave=True)):
        # zero all gradients on each new step
        optimizer.zero_grad()
        optimizer_classifier.zero_grad()

        # prepare batches and more all to the active device
        inputs_ids_a = batch['premise_input_ids'].to(device)
        inputs_ids_b = batch['hypothesis_input_ids'].to(device)
        attention_a = batch['premise_attention_mask'].to(device)
        attention_b = batch['hypothesis_attention_mask'].to(device)
        label = batch['labels'].to(device)

        segment_ids = torch.tensor([0] * max_len).unsqueeze(0).repeat(inputs_ids_a.shape[0], 1).to(device)
        masked_pos = torch.tensor([0] * max_mask).unsqueeze(0).repeat(inputs_ids_a.shape[0], 1).to(device)

        # segment_ids = torch.tensor([0] * max_len).unsqueeze(0).repeat(batch_size, 1).to(device)
        # masked_pos = torch.tensor([0] * max_mask).unsqueeze(0).repeat(batch_size, 1).to(device)

        # # extract token embeddings from BERT at last_hidden_state
        # u, _ = model(inputs_ids_a, segment_ids, masked_pos)
        # v, _ = model(inputs_ids_b, segment_ids, masked_pos)
        # extract token embeddings from BERT at last_hidden_state
        _, _, u = model(inputs_ids_a, segment_ids, masked_pos)
        _, _, v = model(inputs_ids_b, segment_ids, masked_pos)

        u_last_hidden_state = u # all token embeddings A = batch_size, seq_len, hidden_dim
        v_last_hidden_state = v # all token embeddings B = batch_size, seq_len, hidden_dim

         # get the mean pooled vectors
        u_mean_pool = mean_pool(u_last_hidden_state, attention_a) # batch_size, hidden_dim
        v_mean_pool = mean_pool(v_last_hidden_state, attention_b) # batch_size, hidden_dim

        # build the |u-v| tensor
        uv = torch.sub(u_mean_pool, v_mean_pool)   # batch_size,hidden_dim
        uv_abs = torch.abs(uv) # batch_size,hidden_dim

        # concatenate u, v, |u-v|
        x = torch.cat([u_mean_pool, v_mean_pool, uv_abs], dim=-1) # batch_size, 3*hidden_dim

        # process concatenated tensor through classifier_head
        x = classifier_head(x) #batch_size, classifer

        # calculate the 'softmax-loss' between predicted and true label
        loss = criterion(x, label)

        # using loss, calculate gradients and then optimizerize
        loss.backward()
        optimizer.step()
        optimizer_classifier.step()

        scheduler.step() # update learning rate scheduler
        scheduler_classifier.step()

    print(f'Epoch: {epoch + 1} | loss = {loss.item():.6f}')

  0%|          | 0/63 [00:00<?, ?it/s]

Epoch: 1 | loss = 1.274108


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch: 2 | loss = 1.355745


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch: 3 | loss = 1.229014


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch: 4 | loss = 1.048679


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch: 5 | loss = 1.162138


In [40]:
model.eval()
classifier_head.eval()
total_similarity = 0
with torch.no_grad():
    for step, batch in enumerate(eval_dataloader):
        # prepare batches and more all to the active device
        inputs_ids_a = batch['premise_input_ids'].to(device)
        inputs_ids_b = batch['hypothesis_input_ids'].to(device)
        attention_a = batch['premise_attention_mask'].to(device)
        attention_b = batch['hypothesis_attention_mask'].to(device)
        label = batch['labels'].to(device)

        segment_ids = torch.tensor([0] * max_len).unsqueeze(0).repeat(inputs_ids_a.shape[0], 1).to(device)
        masked_pos = torch.tensor([0] * max_mask).unsqueeze(0).repeat(inputs_ids_a.shape[0], 1).to(device)

        # extract token embeddings from BERT at last_hidden_state
        _, _, u = model(inputs_ids_a, segment_ids, masked_pos)  # all token embeddings A = batch_size, seq_len, hidden_dim
        _, _, v = model(inputs_ids_b, segment_ids, masked_pos)  # all token embeddings B = batch_size, seq_len, hidden_dim

        # get the mean pooled vectors
        u_mean_pool = mean_pool(u, attention_a).detach().cpu().numpy().reshape(-1) # batch_size, hidden_dim
        v_mean_pool = mean_pool(v, attention_b).detach().cpu().numpy().reshape(-1) # batch_size, hidden_dim

        similarity_score = cosine_similarity(u_mean_pool, v_mean_pool)
        total_similarity += similarity_score

average_similarity = total_similarity / len(eval_dataloader)
print(f"Average Cosine Similarity: {average_similarity:.4f}")

Average Cosine Similarity: 0.9968


In [42]:
# Save the model after training
torch.save(model.state_dict(), 'models/sbert_model.pt')
print("Model saved to sbert_model.pt")

Model saved to sbert_model.pt


## 7. Inference

In [46]:
import torch
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(model, tokenizer, sentence_a, sentence_b, device):
    # Tokenize and convert sentences to input IDs and attention masks
    inputs_a = tokenizer.encode(sentence_a)
    inputs_b = tokenizer.encode(sentence_b)

    # Move input IDs and attention masks to the active device
    inputs_ids_a = inputs_a['input_ids'][0].unsqueeze(0).to(device)
    attention_a = inputs_a['attention_mask'][0].unsqueeze(0).to(device)
    inputs_ids_b = inputs_b['input_ids'][0].unsqueeze(0).to(device)
    attention_b = inputs_b['attention_mask'][0].unsqueeze(0).to(device)

    # segment_ids = torch.tensor([0] * max_len).unsqueeze(0).repeat(inputs_ids_a.shape[0], 1)
    # masked_pos = torch.tensor([0] * max_mask).unsqueeze(0).repeat(inputs_ids_a.shape[0], 1)

    # Extract token embeddings from BERT
    _, _, u = model(inputs_ids_a, segment_ids, masked_pos)  # all token embeddings A = batch_size, seq_len, hidden_dim
    _, _, v = model(inputs_ids_b, segment_ids, masked_pos)  # all token embeddings B = batch_size, seq_len, hidden_dim

    # Get the mean-pooled vectors
    u = mean_pool(u, attention_a).detach().cpu().numpy().reshape(-1)  # batch_size, hidden_dim
    v = mean_pool(v, attention_b).detach().cpu().numpy().reshape(-1)  # batch_size, hidden_dim

    # Calculate cosine similarity
    similarity_score = cosine_similarity(u.reshape(1, -1), v.reshape(1, -1))[0, 0]

    return similarity_score

# Example usage:
sentence_a = 'Your contribution helped make it possible for us to provide our students with a quality education.'
sentence_b = "Your contributions were of no help with our students' education."
similarity = calculate_similarity(model, tokenizer, sentence_a, sentence_b, device)
print(f"Cosine Similarity: {similarity:.4f}")

Cosine Similarity: 0.9987


In [49]:
import torch
from sklearn.metrics import accuracy_score, classification_report

# Move model and classifier head to evaluation mode
model.eval()
classifier_head.eval()

# Function to get predictions using classifier head
def predict(model, classifier_head, dataset, tokenizer, device):
    predictions = []

    for example in dataset:
        # Tokenize input sentences
        inputs_a = tokenizer.encode(example["premise"])
        inputs_b = tokenizer.encode(example["hypothesis"])

        # Move input IDs and attention masks to the active device
        inputs_ids_a = inputs_a["input_ids"][0].unsqueeze(0).to(device)
        attention_a = inputs_a["attention_mask"][0].unsqueeze(0).to(device)
        inputs_ids_b = inputs_b["input_ids"][0].unsqueeze(0).to(device)
        attention_b = inputs_b["attention_mask"][0].unsqueeze(0).to(device)

        # Define `segment_ids` as a valid tensor (not None)
        segment_ids = torch.zeros((inputs_ids_a.shape[0], inputs_ids_a.shape[1]), dtype=torch.long).to(device)

        # Define `masked_pos` as a valid tensor (not None)
        masked_pos = torch.zeros((inputs_ids_a.shape[0], inputs_ids_a.shape[1]), dtype=torch.long).to(device)

        # Extract embeddings from BERT
        with torch.no_grad():
            _, _, u = model(inputs_ids_a, segment_ids, masked_pos)
            _, _, v = model(inputs_ids_b, segment_ids, masked_pos)

        # Extract last hidden states
        u_last_hidden_state = u  # Shape: (batch_size, seq_len, hidden_dim)
        v_last_hidden_state = v

        # Perform mean pooling to get fixed-size sentence embeddings
        u_mean_pool = mean_pool(u_last_hidden_state, attention_a)
        v_mean_pool = mean_pool(v_last_hidden_state, attention_b)

        # Concatenate u, v, and |u - v| (As used in training)
        x = torch.cat([u_mean_pool, v_mean_pool, torch.abs(u_mean_pool - v_mean_pool)], dim=-1)

        # Pass through classifier head to get logits
        logits = classifier_head(x)

        # Get predicted label
        predicted_label = torch.argmax(logits, dim=1).item()
        predictions.append(predicted_label)

    return predictions

### Matthews Correlation Coefficient & F1-Score for Classification

In [52]:
from sklearn.metrics import accuracy_score, classification_report, matthews_corrcoef, f1_score
import numpy as np

# Get predictions on validation set
predictions = predict(model, classifier_head, raw_dataset['validation'], tokenizer, device)

# Extract true labels
true_labels = np.array([ex['label'] for ex in raw_dataset['validation']])

# Compute Accuracy
accuracy = accuracy_score(true_labels, predictions)

# Compute MCC
mcc = matthews_corrcoef(true_labels, predictions)

# Compute F1-score (macro handles class imbalance better)
f1 = f1_score(true_labels, predictions, average='macro')

# Print all metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")
print(f"F1 Score (Macro): {f1:.4f}")
print("\nClassification Report:\n", classification_report(true_labels, predictions, target_names=["entailment", "neutral", "contradiction"]))

Accuracy: 0.3300
Matthews Correlation Coefficient (MCC): -0.0246
F1 Score (Macro): 0.2526

Classification Report:
                precision    recall  f1-score   support

   entailment       0.34      0.65      0.45       349
      neutral       0.31      0.31      0.31       332
contradiction       0.00      0.00      0.00       319

     accuracy                           0.33      1000
    macro avg       0.22      0.32      0.25      1000
 weighted avg       0.22      0.33      0.26      1000



In [51]:
import pickle

sbert_args = {
    'segment_ids': segment_ids.to('cpu'),
    'masked_pos': masked_pos.to('cpu')
}
pickle.dump(sbert_args, open('models/sbert_args.pkl', 'wb'))

In [58]:
# Define Labels
nli_labels = ["Entailment", "Neutral", "Contradiction"]

def predict_nli(premise, hypothesis):
    """Predicts the NLI label using a custom-trained Sentence Transformer model."""
    model.eval()

    # Tokenize and encode the premise and hypothesis using `.encode()`
    inputs_a = tokenizer.encode([premise])
    inputs_b = tokenizer.encode([hypothesis])

    # Move input IDs and attention masks to the active device
    inputs_ids_a = inputs_a["input_ids"][0].unsqueeze(0).to(device)
    attention_a = inputs_a["attention_mask"][0].unsqueeze(0).to(device)
    inputs_ids_b = inputs_b["input_ids"][0].unsqueeze(0).to(device)
    attention_b = inputs_b["attention_mask"][0].unsqueeze(0).to(device)

    # Define `segment_ids` and `masked_pos` to prevent errors in the embedding layer
    segment_ids = torch.zeros_like(inputs_ids_a, dtype=torch.long).to(device)
    masked_pos = torch.zeros_like(inputs_ids_a, dtype=torch.long).to(device)

    # Extract sentence embeddings
    with torch.no_grad():
        _, _, u = model(inputs_ids_a, segment_ids, masked_pos)
        _, _, v = model(inputs_ids_b, segment_ids, masked_pos)

    # Mean pooling
    u_mean_pool = mean_pool(u, attention_a)
    v_mean_pool = mean_pool(v, attention_b)

    # Concatenate u, v, and |u - v|
    x = torch.cat([u_mean_pool, v_mean_pool, torch.abs(u_mean_pool - v_mean_pool)], dim=-1)

    # Pass through classifier head
    logits = classifier_head(x)

    # Get predicted label
    predicted_label = torch.argmax(logits, dim=1).item()

    return nli_labels[predicted_label]

In [59]:
# Example Test Cases
premise = "A man is playing a guitar on stage."
hypothesis = "The man is performing music."

predicted_label = predict_nli(premise, hypothesis)
print(f"Prediction: {predicted_label}")

Prediction: Entailment
